##Initial configuration

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import os

os.chdir(f'/content/drive/MyDrive/Polimi/NLP/project/t5_generative_model')
os.getcwd()

'/content/drive/MyDrive/Polimi/NLP/project/t5_generative_model'

In [7]:
#!pip install --upgrade numpy
#!pip install --upgrade gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 133.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.5 which is incompatible.
fastai 2.7.19 requires torch<2.7,>=1.10, but you have torch 2.7.0 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.5 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 74.5 MB/s eta 0:00:00
   ━

##Data analysis

###Download of the tools

We download pyterrier to analize the dataset

In [ ]:
!pip install -q python-terrier==0.11.0

Import of pyterrier

In [ ]:
import pyterrier as pt

if not pt.started():
  pt.init()

<ipython-input-10-ff2ffbf0b6dd>:3: DeprecationWarning: Call to deprecated function (or staticmethod) started. (use pt.java.started() instead) -- Deprecated since version 0.11.0.
  if not pt.started():
Java started and loaded: pyterrier.java, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]
<ipython-input-10-ff2ffbf0b6dd>:4: DeprecationWarning: Call to deprecated method pt.init(). Deprecated since version 0.11.0.
java is now started automatically with default settings. To force initialisation early, run:
pt.java.init() # optional, forces java initialisation
  pt.init()


###Load of the dataset

Here we load the training and test set

In [4]:
!pip install datasets

Import of all the libraries

In [11]:
import torch
import pandas as pd
from transformers import T5Tokenizer
from datasets import load_dataset
from datasets import Dataset
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments
import torch
import re
import string


In [12]:
# Load the dataset
dataset = load_dataset("neural-bridge/rag-dataset-12000")

train_set = dataset["train"]
test_set = dataset["test"]

df_train = pd.DataFrame(train_set)
df_test = pd.DataFrame(test_set)

# Remove records with null values or empty/whitespace strings
columns_to_check = ["question", "context", "answer"]

# Drop rows with NaN values
df_train = df_train.dropna(subset=columns_to_check)

# Drop rows with empty or whitespace-only strings
for col in columns_to_check:
    df_train = df_train[df_train[col].str.strip() != ""]

# Reset the index
df_train = df_train.reset_index(drop=True)

##Generative question answering model with context

We will use T5-base as a generative model.

###Tokenizer

First we have to tokenize the dataset

In [14]:
# Load tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Preprocessing function
def preprocess_function(example):
    input_text = "question: " + example["question"] + " context: " + example["context"]
    target_text = example["answer"]

    # Tokenize input (truncate and pad to max length)
    model_inputs = tokenizer(
        input_text,
        max_length=512,
        padding="max_length",
        truncation=True
    )

    # Tokenize target/label (truncate and pad to max length)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            target_text,
            max_length=64,
            padding="max_length",
            truncation=True
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Convert your pandas DataFrame to Hugging Face Dataset
train_dataset = Dataset.from_pandas(df_train)

# Apply tokenization
train_dataset = train_dataset.map(preprocess_function, batched=False)

# Set format for PyTorch
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/9598 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


###Training and testing of T5 model

####Training

In [6]:
small_train_dataset = train_dataset.select(range(500))

In [12]:
# Load smaller model
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Define training arguments
training_args = TrainingArguments(
    output_dir="./t5_qa_model_small",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,
    learning_rate=5e-4,
    num_train_epochs=1,
    weight_decay=0.01,
    save_total_limit=1,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=100,
    report_to="none",
    fp16=True,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    tokenizer=tokenizer,
)

<ipython-input-12-04652d1a3ffb>:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [13]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=63, training_loss=1.3116062709263392, metrics={'train_runtime': 702.6215, 'train_samples_per_second': 0.712, 'train_steps_per_second': 0.09, 'total_flos': 67670900736000.0, 'train_loss': 1.3116062709263392, 'epoch': 1.0})

Now we save the model

In [14]:
torch.save(model.state_dict(), 'model_weights.pth')

####Testing

We load the model

In [15]:
model.load_state_dict(torch.load('model_weights.pth'))

<All keys matched successfully>

We use the test set to generate the answer and see how much good the model is.

In [20]:
# The model must be in evaluation mode
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def generate_answer(question, context):
    input_text = f"question: {question} context: {context}"
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
    input_ids = input_ids.to(device)

    with torch.no_grad():
        output_ids = model.generate(
            input_ids,
            max_length=64,
            num_beams=4,
            early_stopping=True
        )

    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Create a copy of the first n elements of the test set
n = 200
df_test_sample = df_test.head(n).copy()

# Generate the answer
df_test_sample["generated_answer"] = df_test_sample.apply(
    lambda row: generate_answer(row["question"], row["context"]),
    axis=1
)


In [21]:
# Loop through the first m rows in the test set and print the output in the desired format
m = 20
for idx, row in df_test_sample.head(m).iterrows():
    print(f"{idx + 1}")
    print(f"Question: {row['question']}")
    print(f"Target Answer: {row['answer']}")
    print(f"Generated Answer: {row['generated_answer']}")
    print("\n" + "-"*50 + "\n")

1
Question: Who is the music director of the Quebec Symphony Orchestra?
Target Answer: The music director of the Quebec Symphony Orchestra is Fabien Gabel.
Generated Answer: Fabien Gabel is the music director of the Quebec Symphony Orchestra.

--------------------------------------------------

2
Question: Who were the four students of the University of Port Harcourt that were allegedly murdered?
Target Answer: The four students of the University of Port Harcourt that were allegedly murdered were Chiadika Lordson, Ugonna Kelechi Obusor, Mike Lloyd Toku and Tekena Elkanah.
Generated Answer: The four students of the University of Port Harcourt were arraigned on a five-count charge of murder, felony, conspiracy, lynching, and burning.

--------------------------------------------------

3
Question: What did Paul Wall offer to all U.S. Olympic Medalists?
Target Answer: Paul Wall wants to give free gold grills to all U.S. Olympic Medalists.
Generated Answer: Paul Wall offered free Gold Gril

Now let's evaluate automatically the genereted answers

In [23]:
def normalize_text(text):
    """Lowercase, remove punctuation, articles, and extra whitespace."""
    def remove_articles(s):
        return re.sub(r'\b(a|an|the)\b', ' ', s, flags=re.IGNORECASE)

    def white_space_fix(s):
        return ' '.join(s.split())

    def remove_punctuation(s):
        return ''.join(ch for ch in s if ch not in string.punctuation)

    def lower(s):
        return s.lower()

    return white_space_fix(remove_articles(remove_punctuation(lower(text))))

def compute_exact_match(prediction, ground_truth):
    return int(normalize_text(prediction) == normalize_text(ground_truth))

def compute_f1_score(prediction, ground_truth):
    pred_tokens = normalize_text(prediction).split()
    gt_tokens = normalize_text(ground_truth).split()

    common_tokens = set(pred_tokens) & set(gt_tokens)
    num_same = len(common_tokens)

    # Handling case where there are no common tokens or one of the lists is empty
    if len(pred_tokens) == 0 or len(gt_tokens) == 0:
        return int(pred_tokens == gt_tokens)
    if num_same == 0:
        return 0

    precision = num_same / len(pred_tokens)
    recall = num_same / len(gt_tokens)

    # Avoid division by zero when precision + recall is 0
    if precision + recall == 0:
        return 0

    return 2 * (precision * recall) / (precision + recall)

# Assuming df_test_sample is a pandas DataFrame already loaded
df_test_sample["exact_match"] = df_test_sample.apply(
    lambda row: compute_exact_match(row["generated_answer"], row["answer"]),
    axis=1
)

df_test_sample["f1_score"] = df_test_sample.apply(
    lambda row: compute_f1_score(row["generated_answer"], row["answer"]),
    axis=1
)

# Compute the average scores
average_f1 = df_test_sample["f1_score"].mean()

print(f"Average F1 Score: {average_f1:.2f}")

Average F1 Score: 0.54


###T5 without any training

Now let's upload the T5 model again and test how good is the model alone without any training on our dataset

In [15]:
model2 = T5ForConditionalGeneration.from_pretrained("t5-small")

# The model must be in evaluation mode
model2.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model2.to(device)

def generate_answer(question, context):
    input_text = f"question: {question} context: {context}"
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
    input_ids = input_ids.to(device)

    with torch.no_grad():
        output_ids = model2.generate(
            input_ids,
            max_length=64,
            num_beams=4,
            early_stopping=True
        )

    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Create a copy of the first n elements of the test set
n = 10
df_test_sample2 = df_test.head(n).copy()

# Generate the answer
df_test_sample2["generated_answer"] = df_test_sample2.apply(
    lambda row: generate_answer(row["question"], row["context"]),
    axis=1
)

Here there are the answer the model would give

In [16]:
# Loop through the first 10 rows in the test set and print the output in the desired format
for idx, row in df_test_sample2.iterrows():
    print(f"{idx + 1}")
    print(f"Question: {row['question']}")
    print(f"Target Answer: {row['answer']}")
    print(f"Generated Answer: {row['generated_answer']}")
    print("\n" + "-"*50 + "\n")

1
Question: Who is the music director of the Quebec Symphony Orchestra?
Target Answer: The music director of the Quebec Symphony Orchestra is Fabien Gabel.
Generated Answer: Fabien Gabel

--------------------------------------------------

2
Question: Who were the four students of the University of Port Harcourt that were allegedly murdered?
Target Answer: The four students of the University of Port Harcourt that were allegedly murdered were Chiadika Lordson, Ugonna Kelechi Obusor, Mike Lloyd Toku and Tekena Elkanah.
Generated Answer: Chiadika Lordson, Ugonna Kelechi Obusor, Mike Lloyd Toku

--------------------------------------------------

3
Question: What did Paul Wall offer to all U.S. Olympic Medalists?
Target Answer: Paul Wall wants to give free gold grills to all U.S. Olympic Medalists.
Generated Answer: Free Gold Grills

--------------------------------------------------

4
Question: What are the main agricultural products that African countries export to the rest of the world